<img src="../img/logoRB.png" width="100">
<Br>

In [3]:
import pandas as pd
import os
import datetime

    
#  Применение прогноза на предстоящую неделю

## Сбор данных

### Загружаем результат с предыдущего этапа работы - таблицу с коэффициентами по 5 параметрам

In [59]:
res_TT = pd.read_excel('../data/res_TT.xlsx')
res_TT = res_TT.rename(columns={'NORMALIZE':'count','Unnamed: 8':'mean','Unnamed: 9':'sem'})\
               .drop(range(2)).drop(columns=['Unnamed: 0'])
res_TT

,Отделение,деньнедели,сезон,праздник,каникулы,factor,count,mean,sem
2,203 мкр корпус 22,Вс,весна-осень,об,каникулы,теплее -43,3,0.782007,0.040278
3,203 мкр корпус 22,Вс,весна-осень,об,учебный,теплее -43,17,0.81303,0.040836
4,203 мкр корпус 22,Вс,весна-осень,праздник,учебный,теплее -43,1,0.704192,NaN
5,203 мкр корпус 22,Вс,весна-осень,предпр,учебный,теплее -43,1,0.898305,NaN
6,203 мкр корпус 22,Вс,зима,об,каникулы,теплее -43,5,0.819363,0.095532
...,...,...,...,...,...,...,...,...,...
1610,ул. Ф.Попова д. 15,Ср,зима,об,учебный,холоднее -43,3,1.05633,0.028639
1611,ул. Ф.Попова д. 15,Чт,весна-осень,об,учебный,теплее -43,3,1.135339,0.144835
1612,ул. Ф.Попова д. 15,Чт,зима,об,каникулы,теплее -43,4,1.095827,0.093319
1613,ул. Ф.Попова д. 15,Чт,зима,об,учебный,теплее -43,7,1.081203,0.034978


### Загружаем календарь категорий и погоды
из excel файла, он пока подготавливается в ручную

<img src="../img/2023-02-11_20-44-15.png">

In [43]:
#нас интересуют столбцы ['Date','деньнедели','сезон','праздник','каникулы']
#и только прогнозируемый период, в данном случае 7 неделя с  по 
#проверим
days_to_predict = pd.DataFrame({'date':pd.date_range(pd.to_datetime('13-02-2023'),
                                   pd.to_datetime('19-02-2023'))})
days_to_predict['week'] = days_to_predict['date'].dt.isocalendar().week 
days_to_predict['day_name'] = days_to_predict['date'].dt.day_name()
days_to_predict

,date,week,day_name
0,2023-02-13,7,Monday
1,2023-02-14,7,Tuesday
2,2023-02-15,7,Wednesday
3,2023-02-16,7,Thursday
4,2023-02-17,7,Friday
5,2023-02-18,7,Saturday
6,2023-02-19,7,Sunday


In [44]:
man_calendar = pd.read_excel('../data/catalogs/man_calendar.xlsx')
man_calendar_to_predict = man_calendar[man_calendar['Date'].isin(days_to_predict['date'])]\
            [['Date','деньнедели','сезон','праздник','каникулы']]\
            .rename(columns={'Date':'date'})
man_calendar_to_predict

,date,деньнедели,сезон,праздник,каникулы
773,2023-02-13,Пн,зима,предпр,каникулы
774,2023-02-14,Вт,зима,праздник,каникулы
775,2023-02-15,Ср,зима,об,каникулы
776,2023-02-16,Чт,зима,об,каникулы
777,2023-02-17,Пт,зима,об,каникулы
778,2023-02-18,Сб,зима,об,каникулы
779,2023-02-19,Вс,зима,об,каникулы


In [45]:
# Скачиваем данные о погоде, файл из сайта https://meteostat.net/ru/place/ru/yakutsk?s=24959&t=2021-01-01/2023-01-15
weather = pd.read_excel('../data/catalogs/weather_2021-2022.xlsx')

weather['date'] = pd.to_datetime(weather['date'])

#Установка границы температуры
t_key = -43
weather['factor'] = weather['tavg'].map(lambda x: f'холоднее {t_key}' if x<=t_key else f'теплее {t_key}')

# Установка 1 го похолодавшего дня
weather.loc[(weather['factor'].shift(1)==f'теплее {t_key}')&(weather['factor']==f'холоднее {t_key}'),
           'factor'] = f'похолодало {t_key}'

weather.tail(15)

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,factor
763,2023-02-03,-37.5,-41.0,-34.6,0.0,NaN,16.0,7.4,NaN,1015.8,NaN,теплее -43
764,2023-02-04,-36.0,-42.0,-32.5,0.0,NaN,350.0,5.7,NaN,1016.5,NaN,теплее -43
765,2023-02-05,-41.4,-44.0,-37.0,0.0,NaN,215.0,4.2,NaN,1019.1,NaN,теплее -43
766,2023-02-06,-41.0,-45.0,-38.0,0.0,NaN,279.0,5.5,NaN,1024.0,NaN,теплее -43
767,2023-02-07,-43.5,-46.0,-40.0,0.0,NaN,294.0,3.5,NaN,1026.5,NaN,похолодало -43
768,2023-02-08,-45.4,-48.0,-43.0,0.0,NaN,342.0,3.1,NaN,1028.0,NaN,холоднее -43
769,2023-02-09,-45.8,-48.0,-43.0,0.0,NaN,2.0,4.0,NaN,1028.2,NaN,холоднее -43
770,2023-02-10,-43.0,-47.0,-39.7,0.0,NaN,0.0,4.3,NaN,1029.3,NaN,холоднее -43
771,2023-02-11,-39.8,-42.2,-36.4,0.0,NaN,347.0,3.0,NaN,1027.5,NaN,теплее -43
772,2023-02-12,-37.8,-39.7,-34.7,NaN,NaN,324.0,2.5,NaN,1026.6,NaN,теплее -43


In [46]:
#Предварительно соединяем погодный фактор к календарю
man_calendar_to_predict = pd.merge(left=man_calendar_to_predict, 
                                   right=weather[['date','factor']], 
                                   how='left', 
                                   left_on=['date'], right_on=['date'])
man_calendar_to_predict

,date,деньнедели,сезон,праздник,каникулы,factor
0,2023-02-13,Пн,зима,предпр,каникулы,теплее -43
1,2023-02-14,Вт,зима,праздник,каникулы,теплее -43
2,2023-02-15,Ср,зима,об,каникулы,теплее -43
3,2023-02-16,Чт,зима,об,каникулы,теплее -43
4,2023-02-17,Пт,зима,об,каникулы,теплее -43
5,2023-02-18,Сб,зима,об,каникулы,NaN
6,2023-02-19,Вс,зима,об,каникулы,NaN


In [47]:
#не хватает данных о погоде в ручную вводим 'теплее -43' 
man_calendar_to_predict.loc[man_calendar_to_predict['factor'].isna(),'factor'] = 'теплее -43' 
man_calendar_to_predict

,date,деньнедели,сезон,праздник,каникулы,factor
0,2023-02-13,Пн,зима,предпр,каникулы,теплее -43
1,2023-02-14,Вт,зима,праздник,каникулы,теплее -43
2,2023-02-15,Ср,зима,об,каникулы,теплее -43
3,2023-02-16,Чт,зима,об,каникулы,теплее -43
4,2023-02-17,Пт,зима,об,каникулы,теплее -43
5,2023-02-18,Сб,зима,об,каникулы,теплее -43
6,2023-02-19,Вс,зима,об,каникулы,теплее -43


In [ ]:
#Не хватает данных, меняем каникулы на нормальный учебный

In [30]:
#man_calendar_to_predict['каникулы']='учебный'

### Загружаем недельные прогнозные данные по точкам
Это сделано после визуальной оценки динамики по сравнению с прошлыми периодами на дашборде Power BI
<img src="../img/2023-02-11_19-44-33_.png" width="700">

Спрогнозируем на Power BI + excel
<img src="../img/2023-02-11_21-46-52.png" width="700">
<img src="../img/2023-02-11_22-30-21.png" width="700">

In [48]:
man_predict_7week = pd.read_excel('../data/for_pwbi/csvs/man_predict_7week.xlsx', header=3)\
              [['Названия строк',7]].drop(20).rename(columns={'Названия строк':'Отделение'}) 
man_predict_7week['avr_day7week'] = man_predict_7week[7]/7
man_predict_7week['avr_day7week'] = man_predict_7week['avr_day7week'].astype(int)
man_predict_7week

,Отделение,7,avr_day7week
0,203 мкр корпус 22,657.101303,93
1,50 лет октября 6,870.844399,124
2,Автодорожный 44,626.415413,89
3,Алексеева 7,897.297753,128
4,Дзержинского 1,1590.375617,227
5,Дзержинского 30а,1427.422959,203
6,Дзержинского 42 к.3,805.240083,115
7,Кирова 20,1567.096665,223
8,Кузьмина 34а,815.821424,116
9,Ларионова 16,991.471692,141


## Сводим данные

### Подготовим матрицу для прогноза

In [49]:
man_predict_7week['days_to_predict'] = man_predict_7week[7].apply(lambda x: days_to_predict['date'].to_list())

In [50]:
man_predict_7week = man_predict_7week.explode('days_to_predict')

### Мердж

In [51]:
man_predict_7week

,Отделение,7,avr_day7week,days_to_predict
0,203 мкр корпус 22,657.101303,93,2023-02-13
0,203 мкр корпус 22,657.101303,93,2023-02-14
0,203 мкр корпус 22,657.101303,93,2023-02-15
0,203 мкр корпус 22,657.101303,93,2023-02-16
0,203 мкр корпус 22,657.101303,93,2023-02-17
...,...,...,...,...
19,Юнимолл,930.099911,132,2023-02-15
19,Юнимолл,930.099911,132,2023-02-16
19,Юнимолл,930.099911,132,2023-02-17
19,Юнимолл,930.099911,132,2023-02-18


In [60]:
res_TT.head()

,Отделение,деньнедели,сезон,праздник,каникулы,factor,count,mean,sem
2,203 мкр корпус 22,Вс,весна-осень,об,каникулы,теплее -43,3,0.782007,0.040278
3,203 мкр корпус 22,Вс,весна-осень,об,учебный,теплее -43,17,0.81303,0.040836
4,203 мкр корпус 22,Вс,весна-осень,праздник,учебный,теплее -43,1,0.704192,NaN
5,203 мкр корпус 22,Вс,весна-осень,предпр,учебный,теплее -43,1,0.898305,NaN
6,203 мкр корпус 22,Вс,зима,об,каникулы,теплее -43,5,0.819363,0.095532


In [61]:
res = pd.merge(left=pd.merge(left=man_predict_7week,
                        right=man_calendar_to_predict,
                        how='left',
                        left_on='days_to_predict', right_on='date'),
               right=res_TT,
               how='left',
               left_on=['Отделение','деньнедели','сезон','праздник','каникулы','factor'],
               right_on=['Отделение','деньнедели','сезон','праздник','каникулы','factor'])

In [63]:
res

,Отделение,7,avr_day7week,days_to_predict,date,деньнедели,сезон,праздник,каникулы,factor,count,mean,sem,predict
0,203 мкр корпус 22,657.101303,93,2023-02-13,2023-02-13,Пн,зима,предпр,каникулы,теплее -43,NaN,NaN,NaN,NaN
1,203 мкр корпус 22,657.101303,93,2023-02-14,2023-02-14,Вт,зима,праздник,каникулы,теплее -43,NaN,NaN,NaN,NaN
2,203 мкр корпус 22,657.101303,93,2023-02-15,2023-02-15,Ср,зима,об,каникулы,теплее -43,6,1.129124,0.079529,105.00851
3,203 мкр корпус 22,657.101303,93,2023-02-16,2023-02-16,Чт,зима,об,каникулы,теплее -43,7,1.188888,0.066776,110.566587
4,203 мкр корпус 22,657.101303,93,2023-02-17,2023-02-17,Пт,зима,об,каникулы,теплее -43,5,1.112715,0.097215,103.482457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,Юнимолл,930.099911,132,2023-02-15,2023-02-15,Ср,зима,об,каникулы,теплее -43,7,1.051584,0.053511,138.809076
136,Юнимолл,930.099911,132,2023-02-16,2023-02-16,Чт,зима,об,каникулы,теплее -43,8,1.253337,0.100794,165.440533
137,Юнимолл,930.099911,132,2023-02-17,2023-02-17,Пт,зима,об,каникулы,теплее -43,9,1.152432,0.072656,152.121045
138,Юнимолл,930.099911,132,2023-02-18,2023-02-18,Сб,зима,об,каникулы,теплее -43,9,0.983596,0.015194,129.834628


In [62]:
res['predict'] = (res['avr_day7week']*res['mean'])
res['predict'].astype('int',errors='ignore')

0             NaN
1             NaN
2       105.00851
3      110.566587
4      103.482457
          ...    
135    138.809076
136    165.440533
137    152.121045
138    129.834628
139    134.800627
Name: predict, Length: 140, dtype: object

In [64]:
res[res['mean'].isna()]

,Отделение,7,avr_day7week,days_to_predict,date,деньнедели,сезон,праздник,каникулы,factor,count,mean,sem,predict
0,203 мкр корпус 22,657.101303,93,2023-02-13,2023-02-13,Пн,зима,предпр,каникулы,теплее -43,NaN,NaN,NaN,NaN
1,203 мкр корпус 22,657.101303,93,2023-02-14,2023-02-14,Вт,зима,праздник,каникулы,теплее -43,NaN,NaN,NaN,NaN
14,Автодорожный 44,626.415413,89,2023-02-13,2023-02-13,Пн,зима,предпр,каникулы,теплее -43,NaN,NaN,NaN,NaN
15,Автодорожный 44,626.415413,89,2023-02-14,2023-02-14,Вт,зима,праздник,каникулы,теплее -43,NaN,NaN,NaN,NaN
19,Автодорожный 44,626.415413,89,2023-02-18,2023-02-18,Сб,зима,об,каникулы,теплее -43,NaN,NaN,NaN,NaN
20,Автодорожный 44,626.415413,89,2023-02-19,2023-02-19,Вс,зима,об,каникулы,теплее -43,NaN,NaN,NaN,NaN
35,Дзержинского 30а,1427.422959,203,2023-02-13,2023-02-13,Пн,зима,предпр,каникулы,теплее -43,NaN,NaN,NaN,NaN
36,Дзержинского 30а,1427.422959,203,2023-02-14,2023-02-14,Вт,зима,праздник,каникулы,теплее -43,NaN,NaN,NaN,NaN
42,Дзержинского 42 к.3,805.240083,115,2023-02-13,2023-02-13,Пн,зима,предпр,каникулы,теплее -43,NaN,NaN,NaN,NaN
43,Дзержинского 42 к.3,805.240083,115,2023-02-14,2023-02-14,Вт,зима,праздник,каникулы,теплее -43,NaN,NaN,NaN,NaN


In [68]:
res.to_excel('../data/predict7week_bad0.xlsx')

In [69]:
pd.pivot(res, index='Отделение', columns='days_to_predict', values='predict')

days_to_predict,2023-02-13,2023-02-14,2023-02-15,2023-02-16,2023-02-17,2023-02-18,2023-02-19
Отделение,,,,,,,
203 мкр корпус 22,NaN,NaN,105.00851,110.566587,103.482457,87.607614,76.200745
50 лет октября 6,121.52,114.843077,128.348242,139.265279,148.764945,135.426795,133.507658
Автодорожный 44,NaN,NaN,105.627365,109.039318,111.12973,NaN,NaN
Алексеева 7,138.452261,132.824121,147.000863,138.701396,139.689361,117.529971,97.54985
Дзержинского 1,216.835821,164.320896,259.09084,267.535696,268.19174,203.799825,196.042664
Дзержинского 30а,NaN,NaN,224.851923,236.656917,236.940295,202.451092,189.9118
Дзержинского 42 к.3,NaN,NaN,125.807516,121.696122,142.012942,103.951955,84.430871
Кирова 20,178.245675,129.633218,272.430279,286.335183,268.374017,178.329041,136.997093
Кузьмина 34а,117.276479,117.276479,127.420347,134.490302,129.326241,99.156082,110.172512


In [82]:
pd.pivot(res, index='Отделение', columns='days_to_predict', values='count')

days_to_predict,2023-02-13,2023-02-14,2023-02-15,2023-02-16,2023-02-17,2023-02-18,2023-02-19
Отделение,,,,,,,
203 мкр корпус 22,NaN,NaN,6,7,5,6,5
50 лет октября 6,1,1,7,8,9,8,6
Автодорожный 44,NaN,NaN,2,2,1,NaN,NaN
Алексеева 7,1,1,8,9,9,10,8
Дзержинского 1,1,1,8,9,9,10,8
Дзержинского 30а,NaN,NaN,6,7,6,7,6
Дзержинского 42 к.3,NaN,NaN,4,4,2,3,3
Кирова 20,1,1,7,8,9,9,7
Кузьмина 34а,1,1,8,9,9,10,8


### Вариант просчета с игнором каникулов

In [73]:
man_calendar_to_predict_uch = man_calendar_to_predict.copy()
man_calendar_to_predict_uch

,date,деньнедели,сезон,праздник,каникулы,factor
0,2023-02-13,Пн,зима,предпр,каникулы,теплее -43
1,2023-02-14,Вт,зима,праздник,каникулы,теплее -43
2,2023-02-15,Ср,зима,об,каникулы,теплее -43
3,2023-02-16,Чт,зима,об,каникулы,теплее -43
4,2023-02-17,Пт,зима,об,каникулы,теплее -43
5,2023-02-18,Сб,зима,об,каникулы,теплее -43
6,2023-02-19,Вс,зима,об,каникулы,теплее -43


In [74]:
man_calendar_to_predict_uch['каникулы']='учебный'
man_calendar_to_predict_uch

,date,деньнедели,сезон,праздник,каникулы,factor
0,2023-02-13,Пн,зима,предпр,учебный,теплее -43
1,2023-02-14,Вт,зима,праздник,учебный,теплее -43
2,2023-02-15,Ср,зима,об,учебный,теплее -43
3,2023-02-16,Чт,зима,об,учебный,теплее -43
4,2023-02-17,Пт,зима,об,учебный,теплее -43
5,2023-02-18,Сб,зима,об,учебный,теплее -43
6,2023-02-19,Вс,зима,об,учебный,теплее -43


In [75]:
res_uch = pd.merge(left=pd.merge(left=man_predict_7week,
                        right=man_calendar_to_predict_uch,
                        how='left',
                        left_on='days_to_predict', right_on='date'),
               right=res_TT,
               how='left',
               left_on=['Отделение','деньнедели','сезон','праздник','каникулы','factor'],
               right_on=['Отделение','деньнедели','сезон','праздник','каникулы','factor'])

In [78]:
res_uch['predict'] = (res_uch['avr_day7week']*res_uch['mean'])
res_uch

,Отделение,7,avr_day7week,days_to_predict,date,деньнедели,сезон,праздник,каникулы,factor,count,mean,sem,predict
0,203 мкр корпус 22,657.101303,93,2023-02-13,2023-02-13,Пн,зима,предпр,учебный,теплее -43,1,1.007075,NaN,93.658019
1,203 мкр корпус 22,657.101303,93,2023-02-14,2023-02-14,Вт,зима,праздник,учебный,теплее -43,1,0.775943,NaN,72.162736
2,203 мкр корпус 22,657.101303,93,2023-02-15,2023-02-15,Ср,зима,об,учебный,теплее -43,15,1.077323,0.035903,100.191075
3,203 мкр корпус 22,657.101303,93,2023-02-16,2023-02-16,Чт,зима,об,учебный,теплее -43,15,1.145058,0.033688,106.490439
4,203 мкр корпус 22,657.101303,93,2023-02-17,2023-02-17,Пт,зима,об,учебный,теплее -43,14,1.102877,0.056045,102.567591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,Юнимолл,930.099911,132,2023-02-15,2023-02-15,Ср,зима,об,учебный,теплее -43,18,1.006336,0.020803,132.836414
136,Юнимолл,930.099911,132,2023-02-16,2023-02-16,Чт,зима,об,учебный,теплее -43,20,1.000569,0.028173,132.075046
137,Юнимолл,930.099911,132,2023-02-17,2023-02-17,Пт,зима,об,учебный,теплее -43,17,1.133833,0.029945,149.665992
138,Юнимолл,930.099911,132,2023-02-18,2023-02-18,Сб,зима,об,учебный,теплее -43,15,1.070892,0.020591,141.357764


In [79]:
pd.pivot(res_uch, index='Отделение', columns='days_to_predict', values='predict')

days_to_predict,2023-02-13,2023-02-14,2023-02-15,2023-02-16,2023-02-17,2023-02-18,2023-02-19
Отделение,,,,,,,
203 мкр корпус 22,93.658019,72.162736,100.191075,106.490439,102.567591,88.137787,80.456754
50 лет октября 6,97.669691,133.762144,114.931931,119.882346,130.579103,139.617069,139.276889
Автодорожный 44,NaN,NaN,83.46778,121.620126,135.811491,79.97973,99.804079
Алексеева 7,114.434783,113.328063,137.639855,139.573781,143.937913,114.870043,103.827015
Дзержинского 1,237.607477,242.364156,239.87148,249.53603,253.685061,191.668031,183.300766
Дзержинского 30а,163.065574,251.779126,206.826875,218.954331,207.559134,200.202324,180.800372
Дзержинского 42 к.3,NaN,NaN,121.82474,115.12534,124.844901,115.18496,98.559196
Кирова 20,256.283582,189.041966,261.064804,258.435359,260.153002,170.80393,139.669637
Кузьмина 34а,124.819835,94.021053,114.749855,120.94418,129.362261,113.816933,103.133996


In [80]:
pd.pivot(res_uch, index='Отделение', columns='days_to_predict', values='count')

days_to_predict,2023-02-13,2023-02-14,2023-02-15,2023-02-16,2023-02-17,2023-02-18,2023-02-19
Отделение,,,,,,,
203 мкр корпус 22,1,1,15,15,14,12,12
50 лет октября 6,1,1,17,17,15,15,16
Автодорожный 44,NaN,NaN,2,3,2,1,2
Алексеева 7,1,1,19,20,18,16,17
Дзержинского 1,1,1,19,20,18,16,17
Дзержинского 30а,1,1,16,16,14,13,13
Дзержинского 42 к.3,NaN,NaN,7,7,6,5,5
Кирова 20,1,1,18,20,17,16,17
Кузьмина 34а,1,1,19,20,18,16,17


In [81]:
res_uch.to_excel('../data/predict7week_uch.xlsx')

In [3]:

datetime.datetime.now().strftime('%d-%m-%Y')

'11-02-2023'

In [5]:
pd.date_range(pd.to_datetime('13-02-2023'),pd.to_datetime('19-02-2023'))

DatetimeIndex(['2023-02-13', '2023-02-14', '2023-02-15', '2023-02-16',
               '2023-02-17', '2023-02-18', '2023-02-19'],
              dtype='datetime64[ns]', freq='D')

In [28]:
#Создаем календарь дней для прогноза
days_to_predict = pd.DataFrame({'date':pd.date_range(pd.to_datetime('13-02-2023'),
                                                     pd.to_datetime('19-02-2023'))})
days_to_predict['week'] = days_to_predict['date'].dt.isocalendar().week
days_to_predict['day_name'] = days_to_predict['date'].dt.day_name()
days_to_predict['year'] = days_to_predict['date'].dt.year
days_to_predict['month'] = days_to_predict['date'].dt.month
days_to_predict['day'] = days_to_predict['date'].dt.day

In [29]:
days_to_predict

,date,week,day_name,year,month,day
0,2023-02-13,7,Monday,2023,2,13
1,2023-02-14,7,Tuesday,2023,2,14
2,2023-02-15,7,Wednesday,2023,2,15
3,2023-02-16,7,Thursday,2023,2,16
4,2023-02-17,7,Friday,2023,2,17
5,2023-02-18,7,Saturday,2023,2,18
6,2023-02-19,7,Sunday,2023,2,19
